1/23 기준 현재 구현: 소송 유형 분류만 (5개 카테고리)
- 형사소송, 행정소송, 민사/가사소송

필요한 추가 기능:
1. 승소율 예측
2. 형량/벌금 예측 
3. 위험도 분석
4. 관련 판례 추출

In [1]:
# 머신러닝
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from ydata_profiling import ProfileReport

c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2. 데이터 로드 함수 (폴더 구조를 타고 들어가 모든 JSON을 읽음)
def load_legal_data(base_path):
    data_list = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.json'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    try:
                        content = json.load(f)
                        info = content.get('info', {})
                        label = content.get('label', {})
                        data_list.append({
                            'category': info.get('lawClass', ''),    # 법률 분류 코드 (예: 02)
                            'case_num': info.get('caseNum', ''),     # 사건 번호
                            'case_name': info.get('caseName', ''),    # 사건명
                            'case_code': info.get('caseCode', ''),    # 사건 코드 (헌마, 헌바 등)
                            'question': label.get('input', ''),       # 질의 내용 (추가 권장)
                            'body': label.get('output', '')           # 응답 내용
                        })
                    except Exception as e:
                        print(f"Error reading {file}: {e}")
                        continue
    return pd.DataFrame(data_list)

In [3]:
# # 데이터 불러오기(전처리된 피클파일로 로딩/json데이터는 커서 너무 오래걸림.)
print("✅ 전처리된 데이터 로딩 중...")
train_df = pd.read_pickle('../pkl_file/pkl/rowtrain_pklFile.pkl')
test_df = pd.read_pickle('../pkl_file/pkl/rowtest_pklFile.pkl')
print("✅ 전처리된 데이터 로딩 완료")

✅ 전처리된 데이터 로딩 중...
✅ 전처리된 데이터 로딩 완료


In [4]:
# # 3. 데이터 불러오기(피클파일이 대체되어서 실행안함) 가장최초에 실행
# train_df = load_legal_data('../dataset/training')
# print(" 트레이닝 제이슨데이터 로딩 완료")
# test_df = load_legal_data('../dataset/haksup')
# print(" 테스트 제이슨데이터 로딩 완료")

In [5]:
# 데이터 확인
print(f"✅ 로드 완료!")
print(f"학습 데이터: {len(train_df)}건 / 검증 데이터: {len(test_df)}건")
print("\n[컬럼 정보]")
print(train_df.columns)
print("\n[데이터 샘플]")
train_df.head()

✅ 로드 완료!
학습 데이터: 48308건 / 검증 데이터: 5435건

[컬럼 정보]
Index(['category', 'case_num', 'case_name', 'case_code', 'question', 'body',
       'case_type', 'legal_text', 'text'],
      dtype='object')

[데이터 샘플]


,category,case_num,case_name,case_code,question,body,case_type,legal_text,text
78,02,2008헌마205,기소유예처분취소,일반법원,피청구인이 청구인의 신호위반 책임을 인정한 구체적인 증거들은 무엇이었나요?,피청구인은 청구인에게 신호위반의 책임을 인정하기 위해 스타렉스 승합차의 운전자와 탑...,형사소송,기소유예처분취소 피청구인이 청구인의 신호위반 책임을 인정한 구체적인 증거들은 무엇이...,기소유예처분취소 피청구인이 청구인의 신호위반 책임을 인정한 구체적인 증거들은 무엇이...
79,02,2008헌마472,기소유예처분취소,일반법원,헌법재판소는 청구인이 주장한 기소유예처분이 기본권을 침해하지 않았다고 어떻게 판단했나요?,헌법재판소는 청구인이 주장한 기소유예처분이 기본권을 침해하지 않았다고 판단했습니다....,형사소송,기소유예처분취소 헌법재판소는 청구인이 주장한 기소유예처분이 기본권을 침해하지 않았다...,기소유예처분취소 헌법재판소는 청구인이 주장한 기소유예처분이 기본권을 침해하지 않았다...
80,02,2008헌마684,기소유예처분취소,일반법원,기소유예의 불기소처분이 청구인의 기본권을 침해했다고 볼 수 없는 이유는 무엇인가요?,피청구인의 불기소처분이 헌법재판소가 관여할 정도의 자의적인 처분으로 인정될 자료가 ...,형사소송,기소유예처분취소 기소유예의 불기소처분이 청구인의 기본권을 침해했다고 볼 수 없는 이...,기소유예처분취소 기소유예의 불기소처분이 청구인의 기본권을 침해했다고 볼 수 없는 이...
81,02,2008헌마718,기소유예처분취소,일반법원,헌법재판소는 청구인이 기소유예 처분이 헌법상 평등권을 침해했다고 주장했음에도 불구하...,헌법재판소는 피청구인이 청소년보호법 위반 사건에서 기소유예 처분을 내릴 때 헌법 해...,형사소송,기소유예처분취소 헌법재판소는 청구인이 기소유예 처분이 헌법상 평등권을 침해했다고 주...,기소유예처분취소 헌법재판소는 청구인이 기소유예 처분이 헌법상 평등권을 침해했다고 주...
83,02,2009헌마69,구속처분취소,일반법원,이미 헌법재판소로부터 각하된 사건에 대해 다시 심판 청구를 한 경우 어떤 결정이 내...,"구속집행에 대한 위헌확인을 청구하였으나, 이는 이전에 동일 사건에 대해 헌법소원심판...",형사소송,구속처분취소 이미 헌법재판소로부터 각하된 사건에 대해 다시 심판 청구를 한 경우 어...,구속처분취소 이미 헌법재판소로부터 각하된 사건에 대해 다시 심판 청구를 한 경우 어...


In [6]:
# 각 소송 유형별 데이터 개수 확인 / 데이터셋 자료 사용하면 이건 주석처리해야함
print("\n[학습 데이터 소송 유형 분포]")
train_counts = train_df['case_type'].value_counts()
print(train_counts)
print(f"Total: {train_counts.sum()}건") # 전체 합계 출력

print("\n[검증 데이터 소송 유형 분포]")
test_counts = test_df['case_type'].value_counts()
print(test_counts)
print(f"Total: {test_counts.sum()}건") # 전체 합계 출력


[학습 데이터 소송 유형 분포]
case_type
형사소송       37831
행정소송        9433
민사/가사소송     1044
Name: count, dtype: int64
Total: 48308건

[검증 데이터 소송 유형 분포]
case_type
형사소송       4542
행정소송        801
민사/가사소송      92
Name: count, dtype: int64
Total: 5435건


In [7]:
# 라벨링 및 텍스트 통합
def label_case_type(row):
    text = f"{row['case_name']} {row['question']} {row['body']}"

    # 1️⃣ 헌법재판소 사건
    if row['case_code'].startswith('헌'):
        return '헌법'

    # 2️⃣ 형사소송
    criminal_keywords = [
        '사기', '폭행', '상해', '살인', '강도', '절도',
        '처벌', '징역', '벌금', '구속', '기소', '피고인', '유죄'
    ]
    if any(k in text for k in criminal_keywords):
        return '형사소송'

    # # 3️⃣ 가사소송
    # family_keywords = [
    #     '이혼', '양육권', '친권', '위자료', '재산분할', '혼인'
    # ]
    # if any(k in text for k in family_keywords):
    #     return '가사소송'

    # 3️⃣ 행정소송
    admin_keywords = [
        '처분', '취소', '과세', '부과', '행정', '허가',
        '등록', '공무원', '징계'
    ]
    if any(k in text for k in admin_keywords):
        return '행정소송'

    # 4️⃣ 민사소송
    civil_family_keywords = [
        '손해배상', '계약', '채무', '채권',
        '임대차', '소유권', '부당이득','보증금','임차권', '등기명령', '임차권 등기명령',
        '이혼', '양육권', '친권', '위자료', '재산분할', '혼인'
    ]
    if any(k in text for k in civil_family_keywords):
        return '민사/가사소송'

    return '기타'

train_df['case_type'] = train_df.apply(label_case_type, axis=1)
test_df['case_type'] = test_df.apply(label_case_type, axis=1)

print(train_df[['case_name', 'case_code', 'case_type']].head())

   case_name case_code case_type
78  기소유예처분취소      일반법원      형사소송
79  기소유예처분취소      일반법원      형사소송
80  기소유예처분취소      일반법원      형사소송
81  기소유예처분취소      일반법원      형사소송
83    구속처분취소      일반법원      형사소송


In [8]:
# 각 소송 유형별 데이터 개수 확인
print("\n[학습 데이터 소송 유형 분포]")
train_counts = train_df['case_type'].value_counts()
print(train_counts)
print(f"Total: {train_counts.sum()}건") # 전체 합계 출력

print("\n[검증 데이터 소송 유형 분포]")
test_counts = test_df['case_type'].value_counts()
print(test_counts)
print(f"Total: {test_counts.sum()}건") # 전체 합계 출력

# 학습 데이터80,000건48,308건 31,692건약 39.62% 감소
# 검증 데이터10,000건5,435건4,565건약 45.65% 감소


[학습 데이터 소송 유형 분포]
case_type
형사소송       37831
행정소송        9433
민사/가사소송     1044
Name: count, dtype: int64
Total: 48308건

[검증 데이터 소송 유형 분포]
case_type
형사소송       4542
행정소송        801
민사/가사소송      92
Name: count, dtype: int64
Total: 5435건


In [9]:
# 학습 데이터에 대한 상세 리포트 생성
# 데이터에 빈 곳은 없는지, 어떤 단어가 많이 나오는지 리포트로 확인합니다.
profile = ProfileReport(train_df, title="Legal AI Training Data Report")
# profile.to_notebook_iframe() # 주피터 노트북 화면에 바로 출력

# 3. 'row_report.html'이라는 이름으로 파일 저장
profile.to_file("../pkl_file/machineRow_report.html")

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 51068 (\N{HANGUL SYLLABLE IL}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 48152 (\N{HANGUL SYLLABLE BAN}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 48277 (\N{HANGUL SYLLABLE BEOB}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 50896 (\N{HANGUL SYLLABLE WEON}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 44033 (\N{HANGUL SYLLABLE GAG}) missing from 

In [10]:
# 리포트 내용 여기에서 확인 'Overview' / 'Variables'
# 전체적인 요약 정보와 데이터 타입 확인
train_df.info()

# 결측치(누락되거나 비어있는값) 개수 상세 확인
train_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 48308 entries, 78 to 79998
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    48308 non-null  object
 1   case_num    48308 non-null  object
 2   case_name   48308 non-null  object
 3   case_code   48308 non-null  object
 4   question    48308 non-null  object
 5   body        48308 non-null  object
 6   case_type   48308 non-null  object
 7   legal_text  48308 non-null  object
 8   text        48308 non-null  object
dtypes: object(9)
memory usage: 3.7+ MB


category      0
case_num      0
case_name     0
case_code     0
question      0
body          0
case_type     0
legal_text    0
text          0
dtype: int64

In [11]:
# 통계치 및 분포 확인 / 'Statistics' 및 'Histogram' 
# 수치형 데이터의 기술 통계량 (평균, 4분위수 등)
train_df.describe()

,category,case_num,case_name,case_code,question,body,case_type,legal_text,text
count,48308,48308,48308,48308,48308,48308,48308,48308,48308
unique,1,24620,4973,4,20553,48174,3,48284,48284
top,02,,,일반법원,,건축물관리법 제51조 제2항 규정에 따르면 무기 또는 1년 이상의 징역에 처합니다.,형사소송,아동학대범죄의 처벌 등에 관한 특례법에서 고소가 없거나 취소된 경우 아동보호사건으...,아동학대범죄의 처벌 등에 관한 특례법에서 고소가 없거나 취소된 경우 아동보호사건으...
freq,48308,14714,14714,48233,27661,11,37831,2,2


In [12]:
# case_code가 빈 값인 데이터들만 모아서 case_type을 확인
empty_code_df = train_df[train_df['case_code'].str.strip() == ""]
print("사건코드가 없는 데이터의 소송 유형 분포:")
print(empty_code_df['case_type'].value_counts())

사건코드가 없는 데이터의 소송 유형 분포:
Series([], Name: count, dtype: int64)


In [13]:
# 범주형(텍스트) 데이터의 빈도수 확인
print(f"[Category Total: {len(train_df['category']):,}]")
print(train_df['category'].value_counts().apply(lambda x: f"{x:,}"), '\n')  #소송 분류 확인

# 컬럼명이 'law' 또는 'article' 등으로 되어 있을 수 있습니다.
print(f"[Case Type Total: {train_df['case_type'].count():,}]")
print(train_df['case_type'].value_counts().head(10).apply(lambda x: f"{x:,}"), '\n')
# 관련법률 빈도 / 상위 10개만 확인 

print(f"[Case Code Total: {train_df['case_code'].count():,}]")
print(train_df['case_code'].value_counts().head(10).apply(lambda x: f"{x:,}"), '\n')
 #승소/패소 균형(분포)확인

# 사건명 빈도 확인
train_df['case_code'] = train_df['case_code'].replace(['', ' '], '일반법원')

print(f"[Case Code Total: {train_df['case_code'].count():,}]")
print(train_df['case_code'].value_counts().head(10).apply(lambda x: f"{x:,}"), '\n')


# 데이터 컬럼 이름 확인 
print(train_df.columns)

[Category Total: 48,308]
category
02    48,308
Name: count, dtype: object 

[Case Type Total: 48,308]
case_type
형사소송       37,831
행정소송        9,433
민사/가사소송     1,044
Name: count, dtype: object 

[Case Code Total: 48,308]
case_code
일반법원    48,233
각하          60
기각          13
기타           2
Name: count, dtype: object 

[Case Code Total: 48,308]
case_code
일반법원    48,233
각하          60
기각          13
기타           2
Name: count, dtype: object 

Index(['category', 'case_num', 'case_name', 'case_code', 'question', 'body',
       'case_type', 'legal_text', 'text'],
      dtype='object')


In [14]:
# 1. NaN(결측치)이거나 공백만 있는 행을 모두 찾습니다.
empty_code_df = train_df[train_df['case_code'].isna() | (train_df['case_code'].astype(str).str.strip() == "")]

if not empty_code_df.empty:
    print("사건코드가 없는 데이터의 소송 유형 분포:")
    print(empty_code_df['case_type'].value_counts())
else:
    print("확인 결과: 사건코드가 비어 있는 데이터가 하나도 없습니다!")
    
# 전체 행 중 중복된 데이터 개수
train_df.duplicated().sum()

# 법률 본문 중복확인
train_df['body'].duplicated().sum()
# 판결문이 같은 사건이 꽤 있음. -> 실제 판례라는 증거.

확인 결과: 사건코드가 비어 있는 데이터가 하나도 없습니다!


np.int64(134)

In [15]:
# 특정 컬럼(예: 법률 본문/TF-IDF / BERT에서 쓸 통합 텍스트)의 중복 확인
train_df['legal_text'] = (
    train_df['case_name'] + " " +
    train_df['question'] + " " +
    train_df['body']
)

train_df['legal_text'].duplicated().sum()
# 중복 데이터 57개 개념을 더 확실히 익히게 만드는 것

np.int64(24)

In [16]:
# 수치형 컬럼이 2개 미만이므로 상관관계 분석생략하고 
#TF-IDF(어휘빈도의 머신러닝)방식 -> KLUE-BERT(딥러닝) 진행.
# TR-IDF는 TF 단어빈도 하나의 문서 내에서 얼마나 자주 등장하는지
# IDF 전체 문서들중에서 얼마나 드물게 등장하는지

# TF-IDF(핵심단어 뽑기) + Logistic(예측하기) & 문장길이 기반 max_length 산정
# 텍스트를 수치화 -> 데이터 분포를 통해 최적의 길이 정하기.


# 이게바로 데이터 전처리!!!!
# TF-IDF는 단어의 빈도를 계산할때
# 핵심 단어를 쉽게 뽑기 위해 하나의 문장으로 만들어 문서 전체의 맥락을 반영.
train_df['text'] = (
    train_df['case_name'] + " " +
    train_df['question'] + " " +
    train_df['body']
)

test_df['text'] = (
    test_df['case_name'] + " " +
    test_df['question'] + " " +
    test_df['body']
)


# 머신러닝 시작! - 로지스틱 회귀
# (이진 분류, 회귀-> 로지스틱 회귀 모델은 학습을 통해 단어별로 점수(가중치)를 매김)
# 1-1. 문장 길이 기반 max_length 산정 (합친 텍스트 데이터 사용)
def calculate_max_length(texts):
    
    # 단어(공백 기준) 개수 계산
    lengths = [len(text.split()) for text in texts]
    
    # 전체 데이터의 95%를 커버하는 길이를 계산
    max_len = int(np.percentile(lengths, 99.9))

    print(f"추천 max_length (99.9% 커버리지): {max_len}")
    return max_len


# 1-2. TF-IDF + Logistic Regression
# 실제 데이터 적용
max_len_val = calculate_max_length(train_df['text'])

tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(train_df['text'])
y_train = train_df['case_type'] 


lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
lr_model.fit(X_tfidf, y_train)

추천 max_length (99.9% 커버리지): 523


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


머신러닝
TF-IDF를 활용하여 Logistic Regression모델을 사용.
→ 단어에 점수를 매겨서 다 더한 후 높은 점수 소송타입으로 분류

분류되어있는 데이터셋의 텍스트 통합 후 단어 분리 및 파악

In [17]:
# 빠른 재실행을 위하여 임포트 해당 셀에 게시.
from sklearn.pipeline import Pipeline
# Pipeline은 TF-IDF와 모델학습을 하나의 통로로 연결해주는 도구 데이터 누수 방지.


# 머신러닝 모델학습
tfidf_model = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=30000,
        ngram_range=(1, 2),
        min_df=3
    )),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced'  # 불균형 대응
    ))
])

# 머신러닝 모델 학습 시작
print("\n 모델 학습 시작...")
tfidf_model.fit(train_df['text'], train_df['case_type'])
preds = tfidf_model.predict(test_df['text'])

# 결과 리포트 출력
print("\n[통합 후 성능 평가]")
print(classification_report(test_df['case_type'], preds))
# precision(정밀도), recall(재현율), f1-score(조회 평균), Support (실제 데이터갯수)
# accuracy (정확도), macro avg(매크로 평균), weighted avg(가중평균)


 모델 학습 시작...

[통합 후 성능 평가]
              precision    recall  f1-score   support

     민사/가사소송       0.72      0.78      0.75        92
        행정소송       0.84      0.94      0.89       801
        형사소송       0.99      0.97      0.98      4542

    accuracy                           0.96      5435
   macro avg       0.85      0.90      0.87      5435
weighted avg       0.96      0.96      0.96      5435



In [18]:
# %% [학습 결과 샘플 확인]
test_samples = test_df.copy()
test_samples['predicted'] = preds
print("--- 모델의 실제 예측 결과 샘플 (10개) ---")
display(test_samples[['text', 'case_type', 'predicted']].head(10))

# %% [임의의 문장 테스트]
def predict_law_type(my_text):
    # tfidf_model이 이미 위에서 학습되었으므로 바로 사용 가능합니다.
    prediction = tfidf_model.predict([my_text])[0]
    proba = tfidf_model.predict_proba([my_text]).max() 
    print(f"입력 문장: {my_text[:50]}...")
    print(f"예측 결과: {prediction} (확신도: {proba:.2%})")
    print("-" * 30)

# 실제 테스트 실행 (학습이 잘 됐나 확인해보는 단계)
# 아래 문장을 모델이 읽었을때 어떻게 분류하는지 테스트 해보는것. 모델 추론의 예시임.
predict_law_type("피고인은 피해자에게 상해를 입혀 전치 4주의 진단을 받게 하였다.")
predict_law_type("임대인이 보증금을 돌려주지 않아 임차권 등기명령을 신청합니다.")
# 이거는 머신러닝 데이터의 한계로 인하여 잘못 처리되어 나올수 있다고 함.

predict_law_type("행정청의 영업정지 처분이 부당하므로 이를 취소해 주시기 바랍니다.")

--- 모델의 실제 예측 결과 샘플 (10개) ---


,text,case_type,predicted
9,기소유예처분취소 기소유예처분이 청구인의 평등권과 행복추구권을 침해한 이유는 무엇인가...,형사소송,형사소송
10,기소유예처분취소 기소유예처분이 청구인의 기본권을 침해했다고 인정될 만한 중대한 잘못...,형사소송,형사소송
11,재정신청기각결정취소 법원의 재판에 대해 예외적으로 헌법소원심판청구가 허용되는 경우는...,행정소송,행정소송
13,재정신청기각결정취소 원의 재판을 대상으로 하는 헌법소원심판청구가 허용되지 않는 이유...,행정소송,행정소송
18,기소유예처분취소 기소유예처분이 청구인의 기본권을 침해했다고 볼 수 없는 이유는 무엇...,형사소송,형사소송
19,기소유예처분취소 기소유예처분이 헌법상 보장된 기본권을 침해하였다는 주장은 어떻게 판...,형사소송,형사소송
26,기소유예처분취소 업무방해 등에 대한 기소유예처분의 취소를 구하는 심판청구가 기각된 ...,형사소송,형사소송
30,"기소유예처분취소 기소유예처분이 헌법상 기본권을 침해하였다고 주장할 경우, 헌법재판소...",형사소송,형사소송
32,기소유예처분취소 기소유예처분이 헌법소원의 대상이 되기 위해서는 어떤 요건을 충족해야...,형사소송,형사소송
33,기소유예처분취소 기소유예처분이 평등권을 침해했다고 주장한 청구인의 주장이 받아들여지...,형사소송,형사소송


입력 문장: 피고인은 피해자에게 상해를 입혀 전치 4주의 진단을 받게 하였다....
예측 결과: 형사소송 (확신도: 92.59%)
------------------------------
입력 문장: 임대인이 보증금을 돌려주지 않아 임차권 등기명령을 신청합니다....
예측 결과: 형사소송 (확신도: 65.83%)
------------------------------
입력 문장: 행정청의 영업정지 처분이 부당하므로 이를 취소해 주시기 바랍니다....
예측 결과: 행정소송 (확신도: 70.94%)
------------------------------


In [19]:
# 예측결과 샘플 확인
test_samples = test_df.copy()
test_samples['predicted'] = preds
test_samples['correct'] = test_samples['case_type'] == test_samples['predicted']

print("\n[예측 결과 샘플 10개]")
display(test_samples[['text', 'case_type', 'predicted', 'correct']].head(10))

print(f"\n정확도: {test_samples['correct'].mean():.2%}")

# %%
print("\n✅ 모든 작업 완료!")
print("=" * 50)
print("📌 필요한 파일:")
print("  - pkl_file/pkl/train_preprocessed.pkl")
print("  - pkl_file/pkl/test_preprocessed.pkl")
print("=" * 50)


[예측 결과 샘플 10개]


,text,case_type,predicted,correct
9,기소유예처분취소 기소유예처분이 청구인의 평등권과 행복추구권을 침해한 이유는 무엇인가...,형사소송,형사소송,True
10,기소유예처분취소 기소유예처분이 청구인의 기본권을 침해했다고 인정될 만한 중대한 잘못...,형사소송,형사소송,True
11,재정신청기각결정취소 법원의 재판에 대해 예외적으로 헌법소원심판청구가 허용되는 경우는...,행정소송,행정소송,True
13,재정신청기각결정취소 원의 재판을 대상으로 하는 헌법소원심판청구가 허용되지 않는 이유...,행정소송,행정소송,True
18,기소유예처분취소 기소유예처분이 청구인의 기본권을 침해했다고 볼 수 없는 이유는 무엇...,형사소송,형사소송,True
19,기소유예처분취소 기소유예처분이 헌법상 보장된 기본권을 침해하였다는 주장은 어떻게 판...,형사소송,형사소송,True
26,기소유예처분취소 업무방해 등에 대한 기소유예처분의 취소를 구하는 심판청구가 기각된 ...,형사소송,형사소송,True
30,"기소유예처분취소 기소유예처분이 헌법상 기본권을 침해하였다고 주장할 경우, 헌법재판소...",형사소송,형사소송,True
32,기소유예처분취소 기소유예처분이 헌법소원의 대상이 되기 위해서는 어떤 요건을 충족해야...,형사소송,형사소송,True
33,기소유예처분취소 기소유예처분이 평등권을 침해했다고 주장한 청구인의 주장이 받아들여지...,형사소송,형사소송,True



정확도: 96.10%

✅ 모든 작업 완료!
📌 필요한 파일:
  - pkl_file/pkl/train_preprocessed.pkl
  - pkl_file/pkl/test_preprocessed.pkl


In [20]:
# 한글 버젼 위 자료를 %로 바꾸고, 한글번역본 추가

# 1. 리포트를 딕셔너리 형태로 추출
report_dict = classification_report(test_df['case_type'], preds, output_dict=True)

# 2. 데이터프레임 변환 및 가공
report_df = pd.DataFrame(report_dict).transpose()

# 3. accuracy 행의 support가 0이나 잘못된 값으로 나오는 것 방지 (전체 개수 삽입)
total_count = int(report_dict['weighted avg']['support'])
report_df.loc['accuracy', 'support'] = total_count

# 4. 수치 변환 (정확도 등 모든 점수에 100을 곱해 %화)
report_df.iloc[:, :-1] = (report_df.iloc[:, :-1] * 100).round(1)

# 5. 컬럼명 변경 (영어 옆에 한글 병기)
report_df.columns = [
    'precision(정밀도) %', 
    'recall(재현율) %', 
    'f1-score(종합점수) %', 
    'support(데이터수)'
]

# 6. 행 이름 변경 (평균 지표 한글 병기)
report_df = report_df.rename(index={
    'accuracy': 'accuracy (전체 정확도)',
    'macro avg': 'avg (평균)',
    'weighted avg': 'weighted avg (가중 평균)'
})

# 6. 데이터 개수(support)는 정수로 표시
report_df['support(데이터수)'] = report_df['support(데이터수)'].astype(int)

print("\n[통합 후 성능 평가 - % 단위 리포트]")
display(report_df) # 주피터 노트북/코랩에서 표로 출력


[통합 후 성능 평가 - % 단위 리포트]


,precision(정밀도) %,recall(재현율) %,f1-score(종합점수) %,support(데이터수)
민사/가사소송,72.0,78.3,75.0,92
행정소송,84.2,94.0,88.8,801
형사소송,99.0,96.8,97.9,4542
accuracy (전체 정확도),96.1,96.1,96.1,5435
avg (평균),85.1,89.7,87.3,5435
weighted avg (가중 평균),96.4,96.1,96.2,5435


In [21]:
# 학습된 pkl파일 저장
train_df.to_pickle("../pkl_file/pkl/rowtrain_pklFile.pkl")
test_df.to_pickle("../pkl_file/pkl/rowtest_pklFile.pkl")

# 학습후 리포트 저장
profile_after = ProfileReport(train_df, title="Legal AI Training Data Report (After Preprocessing)")
profile_after.to_file("../pkl_file/muchineSuccess_report.html")

print("전처리 완료 및 데이터 저장 성공!")
print("저장 위치: ../dataset/pkl_file/"+ "\n" +
      "- rowtrain_pklFile.pkl"+ "\n" +
      "- rowtest_pklFile.pkl"+ "\n" +
      "- preprocessed_report.html")

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 51068 (\N{HANGUL SYLLABLE IL}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 48152 (\N{HANGUL SYLLABLE BAN}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 48277 (\N{HANGUL SYLLABLE BEOB}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 50896 (\N{HANGUL SYLLABLE WEON}) missing from font(s) Arial.
  plt.savefig(
c:\Users\ASUS\pj_hj\Frondend\Ai\.venv\lib\site-packages\ydata_profiling\visualisation\utils.py:73: UserWarning: Glyph 44033 (\N{HANGUL SYLLABLE GAG}) missing from 

전처리 완료 및 데이터 저장 성공!
저장 위치: ../dataset/pkl_file/
- rowtrain_pklFile.pkl
- rowtest_pklFile.pkl
- preprocessed_report.html


In [22]:
print("필터링된 데이터를 피클 파일로 저장 중...")

# 필터링된 데이터 저장
filtered_train_path = '../pkl_file/pkl/train_filtered.pkl'
filtered_test_path = '../pkl_file/pkl/test_filtered.pkl'

train_df.to_pickle(filtered_train_path)
test_df.to_pickle(filtered_test_path)

print(f"✅ 필터링된 데이터 저장 완료!")
print(f"저장 위치:")
print(f"  - {filtered_train_path}")
print(f"  - {filtered_test_path}")
print(f"\n저장된 데이터 크기:")
print(f"  - 학습 데이터: {len(train_df):,}건")
print(f"  - 검증 데이터: {len(test_df):,}건")

필터링된 데이터를 피클 파일로 저장 중...
✅ 필터링된 데이터 저장 완료!
저장 위치:
  - ../pkl_file/pkl/train_filtered.pkl
  - ../pkl_file/pkl/test_filtered.pkl

저장된 데이터 크기:
  - 학습 데이터: 48,308건
  - 검증 데이터: 5,435건


In [23]:
# 학습완료 알림
print("학습완료!")

학습완료!
